In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from pathlib import Path
import requests
from pprint import pprint
from dataclasses import dataclass

import json
import asyncio

In [ ]:
IMDB_HOST_URL = "https://www.imdb.com"
movies_codes = ["tt7657566"]
movie_code_reviews = {}


def get_key_from_movie_page(code: str):
    # land over a particular movie page and get its pagiination link
    # get the response for a particular movie
    response = requests.get(
        url = f"{IMDB_HOST_URL}/title/{code}/reviews",
        params= {
            "ref_" : "tt_urv"
        }
    )
    response.raise_for_status()

    # convert the response HTML to soup tree
    soup = BeautifulSoup(response.content)

    # get the pagination key
    key = soup.find("div", class_="load-more-data").get("data-key")
    print(key)
    return key


def review_links(code: str, key: str):
    # for a pagination link get all the reviews
    # check if current page limit has not reached to 0
    # made requests with given page key
    response = requests.get(
        url = f"{IMDB_HOST_URL}/title/{code}/reviews/_ajax",
        params = {
            "ref_" : "undefined",
            "paginationKey" : key
        }
    )
    response.raise_for_status()

    soup = BeautifulSoup(response.content)

    # with proper response scrape all the review text along with ratings
    all_links_href = soup.find_all("a", class_="title")

    # get the pagination key for the next page
    key = soup.find("div", class_="load-more-data").get("data-key")        
    return [each_resp.get("href") for each_resp in soup.find_all("a", class_="title")], key


async def get_review_information(review_link: str):
    response = requests.get(
        url = f"{IMDB_HOST_URL}{review_link}",
        params= {
            "ref_" : "tt_urv"
        }
    )
    response.raise_for_status()
    soup = BeautifulSoup(response.content)

    # get various attributes from the page
    response_dict = json.loads(soup.find("script" , type = "application/ld+json").string)
    rating_dict = response_dict.get("reviewRating")
    review_detail = {
        "url" : response.request.url,
        "movie" : soup.title.string.split("Review of")[-1].split("- IMDb")[0].strip(),
        "title" : soup.find("a", class_="title").string.strip(),
        "language" : response_dict.get("inLanguage"),
        "description" : soup.find("div", class_ = "text show-more__control").string,
        "rating" : rating_dict.get("ratingValue") if rating_dict else "",
        "max_rating" : rating_dict.get("bestRating") if rating_dict else "",
        "helpful_review" :  soup.find("div", class_="actions text-muted").text.strip().split("\n")[0]

    }

    return review_detail

movie_reviews = []
review_href = []
for code in movies_codes :
    key = get_key_from_movie_page(code=code)
    page_count = 1
    while page_count >= 1 :
        links_href, key = review_links(
            code = code,
            key = key
        )
        page_count = page_count - 1
        
        review_href.extend(links_href)

In [ ]:
reviews = await asyncio.gather(
    *[
        get_review_information(href) for href in review_href
    ]
)


In [ ]:
reviews[]

In [ ]:
response = requests.get(
    url = "https://www.imdb.com/review/rw7877422/",
    params= {
        "ref_" : "tt_urv"
    }
)
response.raise_for_stat
us()
soup = BeautifulSoup(response.content)

In [ ]:
soup.title.string.split("Review of")[-1].split("- IMDb")[0].strip()

In [ ]:
y = BeautifulSoup(x)

In [ ]:
all = y.find_all("a", class_="title")

In [ ]:
all[0]

In [ ]:
soup.find("div", class_="actions text-muted").text.strip().split("\n")[0]
